In [1]:
from cluster_model import get_dataset, get_transformer, DATASET_NAME, MODEL_NAME, SEED, N_DATASIZE, get_block_out_label, N_BLOCKS
import cluster_model

ds = get_dataset(DATASET_NAME)
model, _ = get_transformer(MODEL_NAME)
shuffled = ds.shuffle(seed=SEED)['train'][:N_DATASIZE]['text']

ds = shuffled
labs = [get_block_out_label(i) for i in range(N_BLOCKS)]

/home/lev/code/research/ai/lattice_cluster_paths/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loaded pretrained model EleutherAI/pythia-70m into HookedTransformer
Moving model to device:  cuda


In [11]:
import importlib
importlib.reload(cluster_model)

decomp = cluster_model.Decomposer(model, ds, labs, similarity_cutoff=19)
decomp.load()

Finding optimal number of clusters and such clusters for layer blocks.0.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.1.hook_resid_post
Loading GMM clusters from cache
Finding optimal number of clusters and such clusters for layer blocks.2.hook_resid_post
Loading GMM clusters from cache
Loading cluster scores from cache


In [12]:
from cluster_model import find_max_weight

find_max_weight(decomp.lattice_scores, K=20)

Most negative absolute value: 1e-06
We have 120 clusters
GOT PATHS [(3465002, [0, 8, 57, 89, 120])]


(None, <networkx.classes.digraph.DiGraph at 0x7be80b516bb0>)

In [13]:
import utils
importlib.reload(utils)
utils.top_k_dag_paths_dynamic(decomp.lattice_scores, k=100_000)

[([-1, 16, 8, 0], 3465.0),
 ([-1, 3, 36, 0], 3135.0),
 ([-1, 20, 27, 0], 2840.0),
 ([-1, 17, 21, 0], 2634.0),
 ([-1, 3, 25, 0], 2538.0),
 ([-1, 4, 6, 0], 2530.0),
 ([-1, 25, 28, 0], 2496.0),
 ([-1, 7, 20, 0], 2318.0),
 ([-1, 11, 5, 0], 2248.0),
 ([-1, 25, 22, 0], 2128.0),
 ([-1, 20, 23, 0], 2070.0),
 ([-1, 3, 10, 0], 2063.0),
 ([-1, 25, 3, 0], 1997.0),
 ([-1, 10, 1, 0], 1984.0),
 ([-1, 3, 23, 0], 1981.0),
 ([-1, 3, 38, 0], 1931.0),
 ([-1, 3, 29, 0], 1902.0),
 ([-1, 33, 5, 0], 1896.0),
 ([-1, 3, 35, 0], 1888.0),
 ([-1, 20, 38, 0], 1874.0),
 ([-1, 3, 21, 0], 1872.0),
 ([-1, 3, 27, 0], 1871.0),
 ([-1, 3, 16, 0], 1870.0),
 ([-1, 16, 29, 0], 1867.0),
 ([-1, 22, 4, 0], 1865.0),
 ([-1, 3, 32, 0], 1863.0),
 ([-1, 3, 11, 0], 1861.0),
 ([-1, 3, 24, 0], 1860.0),
 ([-1, 3, 20, 0], 1859.0),
 ([-1, 3, 17, 0], 1856.0),
 ([-1, 3, 4, 0], 1853.0),
 ([-1, 3, 14, 0], 1853.0),
 ([-1, 3, 18, 0], 1853.0),
 ([-1, 3, 33, 0], 1853.0),
 ([-1, 3, 7, 0], 1852.0),
 ([-1, 3, 22, 0], 1852.0),
 ([-1, 3, 19, 0], 1851.0

In [26]:
from circuitsvis.utils.render import RenderedHTML, render

score_path= [8, 57, 89]
score_path= [10, 63, 89]
score_path = [3, 3, 1]

to_score = [d[:100] for d in ds][:100]
scores = decomp.score(
	to_score,
	score_path=score_path
)
tokens = [[model.tokenizer.decode(t) for t in model.tokenizer(d)['input_ids']] for d in to_score]
# TODO: SORT!

# TODO sep fun
html = render(
        "TextNeuronActivations",
        tokens=tokens,
        activations=[[[[tok]] for tok in s] for s in scores],
        firstDimensionName="Layer",
        secondDimensionName="Neuron",
        firstDimensionLabels=None,
				secondDimensionLabels=None
    )

DS (3, 2009, 512) 2009
(2009, 3, 512) [3, 3, 0]


In [27]:
from IPython.core.display import display, HTML

display(HTML(str(html)))

/tmp/ipykernel_619987/288767411.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
